### Task [99](https://github.com/salgo60/ProjectOutdoorGyms/issues/99) "uppdater Stockholms utegym"
Verkar som det händer lite... 

* API [utegym Stockholm](https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=0eb151d8-2bf9-4a3d-b644-5fb51398779c)
  * [API call](https://apigw.stockholm.se/NoAuth/VirtualhittaserviceDMZ/Rest/serviceunits?&filter%5Bservicetype.id%5D=122&page%5Blimit%5D=1500&page%5Boffset%5D=0&sort=name)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-10 15:31:02.166451


In [2]:
url="https://apigw.stockholm.se/NoAuth/VirtualhittaserviceDMZ/Rest/serviceunits?&filter[servicetype.id]=122&page[limit]=1500&page[offset]=0&sort=name"
import urllib3, json
import pandas as pd
from tqdm import tqdm
http = urllib3.PoolManager()
try:
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
except:
    print (r.status, url)


In [21]:
SthmlStadData = pd.json_normalize(data["data"])
SthmlStadData

,type,id,attributes.name,attributes.location.north,attributes.location.east,attributes.address,attributes.isPublished,attributes.created,attributes.changed,relationships.image.data.type,relationships.image.data.id,relationships.image.links.self,relationships.image.links.related,relationships.serviceType.data.type,relationships.serviceType.data.id,relationships.serviceType.links.self,relationships.serviceType.links.related,relationships.regions.data,links.self
0,serviceunits,12756,Akalla Gårds utegym,6588557,145156,Imatragatan 230,True,2019-08-22,2022-02-23,files,6c9ed8be-3f0d-4c93-af16-ba44de88b9ae,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '8'}]",https://apigw.stockholm.se/noauth/virtualhitta...
1,serviceunits,15335,Alviks strands utegym,6579602,149177,Gustavslundsvägen,True,2022-03-11,2022-03-11,files,88ba9596-00d0-41f3-b10d-a12f384d3015,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '1'}]",https://apigw.stockholm.se/noauth/virtualhitta...
2,serviceunits,14997,Aromen utegym,6576875,151468,Ingenjörsvägen 25,True,2021-01-21,2021-09-24,files,696c2e97-3f81-4efa-86c8-40d7eb8e0010,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '16'}]",https://apigw.stockholm.se/noauth/virtualhitta...
3,serviceunits,12757,Björkhagens utegym,6575372,157321,NaN,True,2019-08-22,2019-10-03,files,620e2190-2758-4c70-ba39-ac9cf2dddb60,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '15'}]",https://apigw.stockholm.se/noauth/virtualhitta...
4,serviceunits,14826,Blåsutparkens utegym och seniorgym,6575262,155159,Blåsutvägen 19,True,2020-08-19,2021-02-08,files,38e34709-972d-4ce1-9c5c-8496342464e2,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '2'}]",https://apigw.stockholm.se/noauth/virtualhitta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,serviceunits,13675,Ärvinge utegym,6587132,146225,NaN,True,2019-10-22,2022-01-18,files,2ac1f6e0-4102-4cb7-ad19-d9a74aaaf91d,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '8'}]",https://apigw.stockholm.se/noauth/virtualhitta...
90,serviceunits,12762,Ågestas utegym,6567465,155986,NaN,True,2019-08-22,2019-10-28,files,c7a2a9b5-adfb-410b-9ce8-de4f864a62b2,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '15'}]",https://apigw.stockholm.se/noauth/virtualhitta...
91,serviceunits,13662,Årstaskogens utegym,6576427,153374,NaN,True,2019-10-22,2021-02-05,files,54f29ec3-a624-46ac-9b31-2c5a6f6c96ab,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,servicetypes,122,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': 

In [22]:
SthmlStadData.to_csv("Stockholmutegym.csv")

### SPARQL
Hämta utegym i WD för Stockholm
* [SPARQL](https://w.wiki/59Mm) Wikidata 

In [8]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

querySthlm = """#title:Outdoor gyms Stockholm in WD
SELECT distinct ?inception ?item ?itemLabel ?www   ?servicenr ?API WHERE {
  ?item wdt:P6104 wd:Q107186275.
  OPTIONAL {?item wdt:P571 ?inception}

  OPTIONAL {?item wdt:P217 ?servicenr}
  BIND(URI(CONCAT("https://apigw.stockholm.se/noauth/virtualhittaservicedmz/rest/serviceunits/",?servicenr)) AS ?API)
                 
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en,sv,de" . 
  }
  
?item wdt:P571 ?date. # date
OPTIONAL { ?item wdt:P18 ?img. }
?item wdt:P131 wd:Q506250.
OPTIONAL { ?item wdt:P856 ?www. }
OPTIONAL { ?item wdt:P625 ?coord. }
OPTIONAL { ?item wdt:P217 ?id. }
} order by desc(?date)"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDtotSthlmOutdoorGym = get_sparql_dataframe(endpoint_url, querySthlm)

WDtotSthlmOutdoorGym


,inception,item,itemLabel,www,servicenr,API
0,2023-01-01T00:00:00Z,http://www.wikidata.org/entity/Q108257729,"Kista äng, outdoor gym",https://vaxer.stockholm/projekt/varbergstoppen...,None,None
1,2022-11-01T00:00:00Z,http://www.wikidata.org/entity/Q108183619,"Väständaparken, outdoor gym",https://vaxer.stockholm/nyheter/2020/06/byggst...,None,None
2,2022-08-01T00:00:00Z,http://www.wikidata.org/entity/Q108180708,Teaterparken outdoor gym,https://vaxer.stockholm/globalassets/omraden/h...,None,None
3,2022-05-01T00:00:00Z,http://www.wikidata.org/entity/Q108209053,"Stora Essinge, outdoor gym",https://vaxer.stockholm/projekt/stora-essingen...,None,None
4,2022-04-22T00:00:00Z,http://www.wikidata.org/entity/Q111944569,Husarviksparken outdoor gym,https://motionera.stockholm/hitta-utegym/utegy...,15425,https://apigw.stockholm.se/noauth/virtualhitta...
...,...,...,...,...,...,...
96,2019-08-22T00:00:00Z,http://www.wikidata.org/entity/Q107208370,"Ågestas, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12762,https://apigw.stockholm.se/noauth/virtualhitta...
97,2019-08-22T00:00:00Z,http://www.wikidata.org/entity/Q107208373,"Östbergas, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12763,https://apigw.stockholm.se/noauth/virtualhitta...
98,2019-08-21T00:00:00Z,http://www.wikidata.org/entity/Q107208313,"Eriksdals, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12752,https://apigw.stockholm.se/noauth/virtualhitta...
99,2019-08-21T00:00:00Z,http://www.wikidata.org/entity/Q107208315,"Farsta, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,12753,https://apigw.stockholm.se/noauth/virtualhitta...


* Looks like we have some planned in WD without "servicenr" --> maybe they need more care  

* Would be great if a planned outdoor gym got a servicenr and also a webpage about an ipject had this servicenr

In [19]:
WDtotSthlmOutdoorGym.isnull().sum()

inception     0
item          0
itemLabel     0
www           0
servicenr    10
API          10
dtype: int64

In [17]:
WDtotSthlmOutdoorGymMissService = WDtotSthlmOutdoorGym[WDtotSthlmOutdoorGym["servicenr"].isna()]

In [18]:
WDtotSthlmOutdoorGymMissService

,inception,item,itemLabel,www,servicenr,API
0,2023-01-01T00:00:00Z,http://www.wikidata.org/entity/Q108257729,"Kista äng, outdoor gym",https://vaxer.stockholm/projekt/varbergstoppen...,None,None
1,2022-11-01T00:00:00Z,http://www.wikidata.org/entity/Q108183619,"Väständaparken, outdoor gym",https://vaxer.stockholm/nyheter/2020/06/byggst...,None,None
2,2022-08-01T00:00:00Z,http://www.wikidata.org/entity/Q108180708,Teaterparken outdoor gym,https://vaxer.stockholm/globalassets/omraden/h...,None,None
3,2022-05-01T00:00:00Z,http://www.wikidata.org/entity/Q108209053,"Stora Essinge, outdoor gym",https://vaxer.stockholm/projekt/stora-essingen...,None,None
11,2021-09-01T00:00:00Z,http://www.wikidata.org/entity/Q108307465,"Kräftriket, outdoor gym",https://www.su.se/medarbetare/r%C3%A5d-st%C3%B...,None,None
12,2021-08-01T00:00:00Z,http://www.wikidata.org/entity/Q108284050,"Gubbängsfältet, outdoor gym",https://www.facebook.com/farstasdf/posts/14711...,None,None
14,2021-07-01T00:00:00Z,http://www.wikidata.org/entity/Q108195585,"Tranebergsängen, outdoor gym",https://vaxer.stockholm/projekt/tranebergsangen/,None,None
22,2020-08-19T00:00:00Z,http://www.wikidata.org/entity/Q107208544,"Dalbottens, outdoor gym",https://motionera.stockholm/hitta-utegym/utegy...,None,None
85,2019-10-21T00:00:00Z,http://www.wikidata.org/entity/Q108159294,"Lappis, outdoor gym",https://sssb.se/bo-hos-oss/utegym/,None,None
86,2019-10-21T00:00:00Z,http://www.wikidata.org/entity/Q108159615,"Strix, outdoor gym",https://sssb.se/bo-hos-oss/utegym/,None,None


In [20]:
#Compare WDtotSthlmOutdoorGym["servicenr"] and SthmlStadData[id]
WDtotSthlmOutdoorGym["servicenr"]

0       None
1       None
2       None
3       None
4      15425
       ...  
96     12762
97     12763
98     12752
99     12753
100    12754
Name: servicenr, Length: 101, dtype: object

In [32]:
SthmlStadDataNew = SthmlStadData.rename(columns={'id': 'servicenr'})
#SthmlStadData["servicenr"]
SthmlStadDataNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   type                                     94 non-null     object
 1   servicenr                                94 non-null     object
 2   attributes.name                          94 non-null     object
 3   attributes.location.north                94 non-null     int64 
 4   attributes.location.east                 94 non-null     int64 
 5   attributes.address                       53 non-null     object
 6   attributes.isPublished                   94 non-null     bool  
 7   attributes.created                       94 non-null     object
 8   attributes.changed                       91 non-null     object
 9   relationships.image.data.type            94 non-null     object
 10  relationships.image.data.id              94 non-null     object


In [33]:
OutdoorGym_diff = pd.merge(SthmlStadDataNew, WDtotSthlmOutdoorGym, how='outer', indicator='Exist')

In [37]:
OutdoorGym_diff["Exist"].value_counts()

both          90
right_only    11
left_only      5
Name: Exist, dtype: int64

In [41]:
MissingGyms = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "left_only"]

In [42]:
MissingGyms

,type,servicenr,attributes.name,attributes.location.north,attributes.location.east,attributes.address,attributes.isPublished,attributes.created,attributes.changed,relationships.image.data.type,...,relationships.serviceType.links.self,relationships.serviceType.links.related,relationships.regions.data,links.self,inception,item,itemLabel,www,API,Exist
1,serviceunits,15335,Alviks strands utegym,6579602.0,149177.0,Gustavslundsvägen,True,2022-03-11,2022-03-11,files,...,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '1'}]",https://apigw.stockholm.se/noauth/virtualhitta...,NaN,NaN,NaN,NaN,NaN,left_only
25,serviceunits,12769,Hellasgårdens utegym,6574972.0,159064.0,NaN,True,2019-08-27,2019-10-03,files,...,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '15'}]",https://apigw.stockholm.se/noauth/virtualhitta...,NaN,NaN,NaN,NaN,NaN,left_only
44,serviceunits,15208,Lilla Blecktornsparkens utegym,6577084.0,154785.0,NaN,True,2021-10-26,2021-10-26,files,...,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '12'}]",https://apigw.stockholm.se/noauth/virtualhitta...,NaN,NaN,NaN,NaN,NaN,left_only
73,serviceunits,14824,Svampvägenparkens utegym,6574032.0,154594.0,Mellanvägen 9-11,True,2020-08-19,2020-08-19,files,...,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '2'}]",https://apigw.stockholm.se/noauth/virtualhitta...,NaN,NaN,NaN,NaN,NaN,left_only
88,serviceunits,15299,Älvsjöskogens lilla utegym,6572194.0,149807.0,Älvsjöskogens motionsspår - utmed järnvägsspåret,True,2022-02-11,2022-02-11,files,...,https://apigw.stockholm.se/noauth/virtualhitta...,https://apigw.stockholm.se/noauth/virtualhitta...,"[{'type': 'regions', 'id': '16'}]",https://apigw.stockholm.se/noauth/virtualhitta...,NaN,NaN,NaN,NaN,NaN,left_only


In [43]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-05-10 16:46:15.207875
Time elapsed (hh:mm:ss.ms) 1:15:13.042530
